In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt 

import torch 
from torch import nn 
from torch import optim 
import torch.nn.functional as F
from torch.autograd import Variable 
from torchvision import datasets, transforms, models 

In [4]:
data_dir = 'dogs-vs-cats'

# TODO: define transforms for the training data and testing data 
train_transform = transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                                      transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225)),
                               transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize(255),
                                     transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225)),
                               transforms.ToTensor()])

In [8]:
train_data = datasets.ImageFolder(data_dir + '-train', transform = train_transform)
test_data = datasets.ImageFolder(data_dir + '-test', transform = test_transform)

In [12]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 32)


In [13]:
model = models.densenet121(pretrained = True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/yujinchung/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:11<00:00, 2747458.45it/s]


In [14]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [15]:
# model = features + classifier
# ablove model will work very well on detecting features, 
# but for the classifier part, we have to replace this,
# since the above network has been trainded on ImageNet dataset.

In [18]:
# Freeze parameters so we dont' backprop through them (established.!)
for param in model.parameters():
    param.requres_grad = False 
    
from collections import OrderedDict 
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(500, 2)),
    ('output', nn.LogSoftmax(dim = 1))
]))

model.classifier = classifier

In [19]:
import time 

In [21]:
for device in ['cpu', 'cuda']:
    criterion = nn.NLLLoss() 
    # only train the classifier parameters, feature parameters are frozen 
    optimizer = optim.Adam(model.classifier.parameters(), lr = 0.001) 
    model.to(device) 
    
    for ii, (inputs, labels) in enumerate(trainloader):
        #Move input and label tensors to the GPU 
        inputs, labels = inputs.to(device), labels.to(device) 
        
        start = time.time() 
        outputs = model.forward(inputs) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        
        if ii == 3:
            break 
            
    print(f"Device = {device}; TIme per batch: {(time.time() - start)/3:.3f} seconds")

KeyboardInterrupt: 